In [ ]:
!pip install -q nlu

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece

In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
import pandas as pd
from googletrans import Translator
import re
from textblob import TextBlob
import seaborn as sns
%matplotlib inline
from transformers import pipeline
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS
from collections import defaultdict

In [ ]:
df=pd.read_csv('NLP_Project - Sheet9.csv')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
translator = Translator()

def translate_to_english(row):
    if row['Language'] == 'Marathi':
        translation = translator.translate(row['Article'], src='mr', dest='en')
        return translation.text
    elif row['Language'] == 'Telugu':
        translation = translator.translate(row['Article'], src='te', dest='en')
        return translation.text
    elif row['Language'] == 'Tamil':
        translation = translator.translate(row['Article'], src='ta', dest='en')
        return translation.text
    else:
        return row['Article']

df['Translated_Article'] = df.apply(translate_to_english, axis=1)

In [ ]:
df

In [ ]:
def clean_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    return text

# Applying the clean_text function
df['Translated_Article'] = df['Translated_Article'].apply(clean_text)
df

In [ ]:
df_mum=df.loc[df['City'] == 'Mumbai']
df_mum

In [ ]:
def show_wordcloud(data, title = None):
    stopwords = set(STOPWORDS)
    stopwords.update(["mumbai","India", "said", "year", "say",'maharashtra','state','climate','change','says','city','chennai','hyderabad','days','cities','telangana'])
    wordcloud = WordCloud(
        stopwords=stopwords,
        background_color='black'
).generate(" ".join(data))

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
# Wordcloud for Articles From Mumbai.
show_wordcloud(df_mum["Translated_Article"].values)

In [ ]:
df_Chen=df.loc[df['City'] == 'Chennai']
df_Chen

In [ ]:
show_wordcloud(df_Chen["Translated_Article"].values)

In [ ]:
df_hyd=df.loc[df['City'] == 'Hyderabad']
df_hyd

In [ ]:
show_wordcloud(df_hyd["Translated_Article"].values)

In [ ]:
#TextBlob for polarity and subjectivity
polarity=[]
subjectivity=[]
for txt in df['Translated_Article']:
    blobSenti=TextBlob(txt)
    polarity.append(blobSenti.sentiment.polarity)
    subjectivity.append(blobSenti.sentiment.subjectivity)
df['Polarity']=polarity
df['Subjectivity']=subjectivity


In [ ]:
df

In [ ]:
plt.figure(figsize=(5,6))
sns.boxplot(data=df, x="City", y="Polarity", hue="City").set(title='Polarity across Cities')
plt.show()



In [ ]:
plt.figure(figsize=(5,6))
sns.boxplot(data=df, x="City", y="Subjectivity", hue="City").set(title='Subjectivity across Cities')
plt.show()

In [ ]:
classifier=pipeline("zero-shot-classification")

In [ ]:
labels = ['positive', 'neutral', 'negative']
posNeuNeg = defaultdict(list)

cities = ['Mumbai', 'Hyderabad', 'Chennai']

for city in cities:
    sequence = " ".join(df.loc[df['City'] == city]['Translated_Article'])
    scores = classifier(sequence, labels)['scores']
    posNeuNeg[city].append(scores)




In [ ]:
posNeuNeg

In [ ]:
pnn = pd.DataFrame.from_dict({city: values[0] for city, values in posNeuNeg.items()}, orient='index', columns=['Positive', 'Negative', 'Neutral'])

In [ ]:
pnn

In [ ]:
pnn.plot(kind='bar', rot=0, figsize=(10, 6), colormap='viridis')
plt.title('Positive, Negative and Neutral Scores by City')
plt.ylabel('Sentiment Score')
plt.xlabel('City')
plt.show()

In [ ]:
labels=['angry','fear','happy','sad','surprise']
emotions=defaultdict(list)


for city in cities:
    sequence = " ".join(df.loc[df['City'] == city]['Translated_Article'])
    scores = classifier(sequence, labels)['scores']
    emotions[city].append(scores)



In [ ]:
emotions

In [ ]:
emo = pd.DataFrame.from_dict({city: values[0] for city, values in emotions.items()}, orient='index', columns=['angry','fear','happy','sad','surprise'])

In [ ]:
emo

In [ ]:
# Plotting
emo.plot(kind='bar', rot=0, figsize=(10, 6), colormap='viridis')
plt.title('Emotion Scores by City')
plt.ylabel('Sentiment Score')
plt.xlabel('City')
plt.show()

In [ ]:
labels=['Optimism','Injustice','disgust']
intent=defaultdict(list)


for city in cities:
    sequence = " ".join(df.loc[df['City'] == city]['Translated_Article'])
    scores = classifier(sequence, labels)['scores']
    intent[city].append(scores)


In [ ]:
intt = pd.DataFrame.from_dict({city: values[0] for city, values in intent.items()}, orient='index', columns=labels)

In [ ]:
intt

In [ ]:
# Plotting
intt.plot(kind='bar', rot=0, figsize=(10, 6), colormap='viridis')
plt.title('Intent Scores by City')
plt.ylabel('Sentiment Score')
plt.xlabel('City')
plt.show()

In [ ]:

labels=['Cause','Impact','Mitigation','Adaption']
aspect=defaultdict(list)


for city in cities:
    sequence = " ".join(df.loc[df['City'] == city]['Translated_Article'])
    scores = classifier(sequence, labels)['scores']
    aspect[city].append(scores)

In [ ]:
asp = pd.DataFrame.from_dict({city: values[0] for city, values in aspect.items()}, orient='index', columns=labels)

In [ ]:
asp

In [ ]:
asp.plot(kind='bar', rot=0, figsize=(10, 6), colormap='viridis')
plt.title('Aspect Scores by City')
plt.ylabel('Sentiment Score')
plt.xlabel('City')
plt.show()

In [ ]:
labels=['Informed','Uninformed','Misinformed']
awarness=defaultdict(list)


for city in cities:
    sequence = " ".join(df.loc[df['City'] == city]['Translated_Article'])
    scores = classifier(sequence, labels)['scores']
    awarness[city].append(scores)

In [ ]:
awar = pd.DataFrame.from_dict({city: values[0] for city, values in awarness.items()}, orient='index', columns=labels)

In [ ]:
awar.plot(kind='bar', rot=0, figsize=(10, 6), colormap='viridis')
plt.title('Awarness Scores by City')
plt.ylabel('Sentiment Score')
plt.xlabel('City')
plt.show()

In [ ]:
labels=['Liberal','Conservative']
ideology=defaultdict(list)


for city in cities:
    sequence = " ".join(df.loc[df['City'] == city]['Translated_Article'])
    scores = classifier(sequence, labels)['scores']
    ideology[city].append(scores)

In [ ]:
ideology

In [ ]:
idg = pd.DataFrame.from_dict({city: values[0] for city, values in ideology.items()}, orient='index', columns=labels)

In [ ]:
idg

In [ ]:
idg.plot(kind='bar', rot=0, figsize=(10, 6), colormap='viridis')
plt.title('Sentiment Scores by City')
plt.ylabel('Sentiment Score')
plt.xlabel('City')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 12))

pnn.plot(ax=axes[0, 0],kind='bar', rot=0, colormap='viridis')
axes[0, 0].set_title('PosNegNeu')

emo.plot(ax=axes[0, 1],kind='bar', rot=0, colormap='viridis')
axes[0, 1].set_title('Emotions')


intt.plot(ax=axes[1, 0],kind='bar', rot=0, colormap='viridis')
axes[1, 0].set_title('Intent')

asp.plot(ax=axes[1, 1],kind='bar', rot=0, colormap='viridis')
axes[1, 1].set_title('Aspect')

plt.tight_layout()

plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,7))  # Define 1 row and 2 columns of subplots

awar.plot(ax=axes[0], kind='bar', rot=0, colormap='viridis')
axes[0].set_title('Awareness')

idg.plot(ax=axes[1], kind='bar', rot=0, colormap='viridis')
axes[1].set_title('Ideology')

plt.tight_layout()

plt.show()

#Non Parametric Test ( Kruskal-Wallis H test )

In [ ]:
Final=pd.concat([pnn,emo,intt,asp,awar,idg], axis=1).T

In [ ]:
Final

In [ ]:
Final.describe()

In [ ]:
import scipy.stats as stats

# Perform the Kruskal-Wallis H test
statistic, p_value = stats.kruskal(Final['Mumbai'],Final['Hyderabad'], Final['Chennai'])

# Output the results
print("Kruskal-Wallis H Test:")
print("H-statistic =", statistic)
print("P-value =", p_value)

# Check for statistical significance
alpha = 0.05
if p_value < alpha:
    print("The differences between groups are statistically significant.")
else:
    print("No significant differences between groups were found.")
